In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install -q trl
!pip install -q evaluate rouge_score
!pip install -q torch
!pip install -q tensorboard

In [2]:
import os
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import evaluate

/opt/conda/envs/pytorch/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
# The model that you want to train from the Hugging Face hub
model_name = "davezack/llama-2-7b-npc"

# The instruction dataset to use
dataset_name = "amaydle/npc-dialogue"

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [5]:
train_data = load_dataset(dataset_name, split = "train")
test_data = load_dataset(dataset_name, split = "test")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1723 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/192 [00:00<?, ? examples/s]

In [6]:
prefix = 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n'
delimeter = '###'
instruction = ' Instruction:\n'
input = ' Input:\n'
name = 'Name: '
bio = ', Biography: '
response = ' Response: '

In [7]:
spongebob = pd.DataFrame({'Name': ['Spongebob Squarepants','Spongebob Squarepants','Spongebob Squarepants','Spongebob Squarepants'],
                          'Biography': ['A square yellow sponge named SpongeBob SquarePants lives in a pineapple with his pet snail, Gary, in the city of Bikini Bottom on the floor of the Pacific Ocean. He works as a fry cook at the Krusty Krab. During his time off, SpongeBob has a knack for attracting trouble with his starfish best friend, Patrick. Arrogant octopus Squidward Tentacles, SpongeBob’s neighbor, dislikes SpongeBob because of his childlike behavior.',
                                        'A square yellow sponge named SpongeBob SquarePants lives in a pineapple with his pet snail, Gary, in the city of Bikini Bottom on the floor of the Pacific Ocean. He works as a fry cook at the Krusty Krab. During his time off, SpongeBob has a knack for attracting trouble with his starfish best friend, Patrick. Arrogant octopus Squidward Tentacles, SpongeBob’s neighbor, dislikes SpongeBob because of his childlike behavior.',
                                        'A square yellow sponge named SpongeBob SquarePants lives in a pineapple with his pet snail, Gary, in the city of Bikini Bottom on the floor of the Pacific Ocean. He works as a fry cook at the Krusty Krab. During his time off, SpongeBob has a knack for attracting trouble with his starfish best friend, Patrick. Arrogant octopus Squidward Tentacles, SpongeBob’s neighbor, dislikes SpongeBob because of his childlike behavior.',
                                        'A square yellow sponge named SpongeBob SquarePants lives in a pineapple with his pet snail, Gary, in the city of Bikini Bottom on the floor of the Pacific Ocean. He works as a fry cook at the Krusty Krab. During his time off, SpongeBob has a knack for attracting trouble with his starfish best friend, Patrick. Arrogant octopus Squidward Tentacles, SpongeBob’s neighbor, dislikes SpongeBob because of his childlike behavior.'],
                          'Query': ['What color are you?', 'Who is your pet snail?', 'Where do you work?', 'Who is you best friend?']})

In [8]:
spongebob_list = [prefix + delimeter + instruction + q + '\n\n' + delimeter for q in spongebob['Query']]
spongebob_list_2 = [input + name + n + bio + b for (n, b) in zip(spongebob['Name'], spongebob['Biography'])]
spongebob_text = [x + y for x, y in zip(spongebob_list, spongebob_list_2)]

In [10]:
spongebob['text'] = spongebob_text

In [11]:
spongebob.head()

,Name,Biography,Query,text
0,Spongebob Squarepants,A square yellow sponge named SpongeBob SquareP...,What color are you?,"Below is an instruction that describes a task,..."
1,Spongebob Squarepants,A square yellow sponge named SpongeBob SquareP...,Who is your pet snail?,"Below is an instruction that describes a task,..."
2,Spongebob Squarepants,A square yellow sponge named SpongeBob SquareP...,Where do you work?,"Below is an instruction that describes a task,..."
3,Spongebob Squarepants,A square yellow sponge named SpongeBob SquareP...,Who is you best friend?,"Below is an instruction that describes a task,..."


In [7]:
test_list = [prefix + delimeter + instruction + q + '\n\n' + delimeter for q in test_data['Query']]
test_list_2 = [input + name + n + bio + b for (n, b) in zip(test_data['Name'], test_data['Biography'])]

In [8]:
text = [x + y for x, y in zip(test_list, test_list_2)]

In [9]:
test_data = test_data.add_column('text', text)

In [10]:
print(test_data['text'][0])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is the biggest challenge you face as a teacher?

### Input:
Name: Naina Mathur, Biography: Naina Mathur is a determined and passionate teacher who has a stutter.


In [12]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Your GPU supports bfloat16: accelerate training with bf16=True


config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [13]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

In [15]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
# prompt = "Instruction:\nWhere is the magic key?\n\n ### Input:\n Name: Bikram, Biography: Bikram is a rough and tough smuggler from the streets of Calcutta, India."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)
responses = pipe(list(spongebob['text']))

[[{'generated_text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat color are you?\n\n### Input:\nName: Spongebob Squarepants, Biography: A square yellow sponge named SpongeBob SquarePants lives in a pineapple with his pet snail, Gary, in the city of Bikini Bottom on the floor of the Pacific Ocean. He works as a fry cook at the Krusty Krab. During his time off, SpongeBob has a knack for attracting trouble with his starfish best friend, Patrick. Arrogant octopus Squidward Tentacles, SpongeBob’s neighbor, dislikes SpongeBob because of his childlike behavior.\n\n### Response: Yellow, of course!\n\n### Response: SpongeBob SquarePants!'}],
 [{'generated_text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWho is your pet sn

In [17]:
output = {'instruction': [], 'responses': []}
for x in responses:
    i = x[0]['generated_text'].split('### Response: ')[0]
    r = x[0]['generated_text'].split('### Response: ')[1:]

    output['instruction'].append(i)
    output['responses'].append(r)

In [20]:
output['instruction']

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat color are you?\n\n### Input:\nName: Spongebob Squarepants, Biography: A square yellow sponge named SpongeBob SquarePants lives in a pineapple with his pet snail, Gary, in the city of Bikini Bottom on the floor of the Pacific Ocean. He works as a fry cook at the Krusty Krab. During his time off, SpongeBob has a knack for attracting trouble with his starfish best friend, Patrick. Arrogant octopus Squidward Tentacles, SpongeBob’s neighbor, dislikes SpongeBob because of his childlike behavior.\n\n',
 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWho is your pet snail?\n\n### Input:\nName: Spongebob Squarepants, Biography: A square yellow sponge named SpongeBob SquarePants liv

In [19]:
output['responses']

[['Yellow, of course!\n\n', 'SpongeBob SquarePants!'],
 ['Gary the snail, my trusty sidekick.\n\n',
  'SpongeBob SquarePants, a yellow sponge who lives in a pineapple under the sea.\n\n',
  'I love making people laugh, I’m a f'],
 ['The Krusty Krab, where I work.\n\n',
  'Bikini Bottom, where I live.\n\n',
  'Squidward Tentacles, my neighbor.\n\n\n',
  'I love to go on adventures with my best'],
 ['My best friend is Patrick Star.\n\n',
  "I don't have any enemies.\n\n",
  "I'm always happy.\n\n",
  "I don't have any fears.\n\n\n\n\n\n\n\n\n\n\n\n"]]

In [16]:
output = pd.DataFrame(data = output)

output.to_csv('Question_Response_Output_Updated.csv')

In [17]:
rouge = evaluate.load('rouge')
predictions = [x[0] for x in output['responses']]
references = test_data['Response']
results = rouge.compute(predictions=predictions,
                      references=references)

In [18]:
results

{'rouge1': 0.25090878891165724,
 'rouge2': 0.08941420473222719,
 'rougeL': 0.21676198487332599,
 'rougeLsum': 0.2165993492751515}